In [1]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

from PIL import Image

import glob
import os
import re

cwd = os.getcwd()

%matplotlib inline

In [3]:
SAMPLE_SIZE = 5090
SPLIT_RATIO = 0.7

IMAGE_SIZE = 256
BATCH_SIZE = 16

steps_per_epoch = 3000
validation_steps = 1082
epochs = 3

In [4]:
index = np.arange(SAMPLE_SIZE) + 1
np.random.seed(1234)
np.random.shuffle(index)

split = (int) (SAMPLE_SIZE * SPLIT_RATIO)
x_train_idx = index[0:split]
x_test_idx = index[split:]

In [5]:
def read_image(image_file):
    image = Image.open(image_file).convert('L')
    image = image.convert()
    image = image.resize((IMAGE_SIZE, IMAGE_SIZE))
    image = np.asarray(image).reshape((IMAGE_SIZE, IMAGE_SIZE, 1))
    return 1 - (image / 255)

In [6]:
def input_generator(indexs):
    files = glob.glob("./data/*")
    
    while(True):
        x_image = []
        y_image = []
        
        for idx in indexs:
            picture_files = list(filter(re.compile(".\/data\/sample-" + str(idx) + "-\d+.jpg").search, files))
            
            for picture in picture_files:
                image = read_image(picture)
                picture = picture.split(".")
                image_blank = read_image("." + picture[1] + "-blank." + picture[2])
                
                x_image.append(image)
                y_image.append(image_blank)

                if len(y_image) >= BATCH_SIZE:
                    yield np.array(x_image), np.array(y_image)
                    x_image = []
                    y_image = []

In [7]:
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Reshape, Conv2DTranspose, UpSampling2D, BatchNormalization
from keras.optimizers import RMSprop
from keras.models import Model

Using TensorFlow backend.


In [8]:
def generate_model_v1():
    image_input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1))
    
    img = Conv2D(32, (3, 3), padding='same', activation='relu')(image_input)
    img = Conv2D(32, (3, 3), padding='same', activation='relu')(img)
    
    img = Conv2DTranspose(32, (3, 3), padding='same', activation='relu')(img)
    img = Conv2DTranspose(32, (3, 3), padding='same', activation='relu')(img)
    img = Conv2DTranspose(1, (3, 3), padding='same', activation='sigmoid')(img)

    outputs = img

    optimizer = RMSprop()
    model = Model(inputs=image_input, outputs=outputs)
    model.compile(optimizer=optimizer,
                  loss='mse')
    
    return model

In [9]:
def generate_model_v2():
    image_input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1))
    
    img = Conv2D(32, (3, 3), padding='same', activation='relu')(image_input)
    img = Conv2D(32, (3, 3), padding='same', activation='relu')(img)
    img = MaxPooling2D()(img)
    
    img = Conv2D(64, (3,3), padding='same', activation='relu')(img)
    
    img = Conv2DTranspose(64, (3,3), padding='same', activation='relu')(img)
    img = UpSampling2D()(img)
    
    img = Conv2DTranspose(32, (3, 3), padding='same', activation='relu')(img)
    img = Conv2DTranspose(32, (3, 3), padding='same', activation='relu')(img)
    img = Conv2DTranspose(1, (3, 3), padding='same', activation='sigmoid')(img)

    outputs = img

    optimizer = RMSprop()
    model = Model(inputs=image_input, outputs=outputs)
    model.compile(optimizer=optimizer,
                  loss='mse')
    
    return model

In [10]:
def generate_model_v3():
    image_input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1))
    
    img = Conv2D(32, (3, 3), padding='same', activation='relu', trainable=False)(image_input)
    img = Conv2D(32, (3, 3), padding='same', activation='relu', trainable=False)(img)
    
    img = MaxPooling2D()(img)
    img = Conv2D(64, (3,3), padding='same', activation='relu', trainable=False)(img)
    
    img = MaxPooling2D()(img)
    img = Conv2D(92, (3,3), padding='same', activation='relu')(img)
    
    img = Conv2DTranspose(92, (3,3), padding='same', activation='relu')(img)
    img = UpSampling2D()(img)
    
    img = Conv2DTranspose(64, (3,3), padding='same', activation='relu', trainable=False)(img)
    img = UpSampling2D()(img)
    
    img = Conv2DTranspose(32, (3, 3), padding='same', activation='relu', trainable=False)(img)
    img = Conv2DTranspose(32, (3, 3), padding='same', activation='relu', trainable=False)(img)
    img = Conv2DTranspose(1, (3, 3), padding='same', activation='sigmoid', trainable=False)(img)

    outputs = img

    optimizer = RMSprop()
    model = Model(inputs=image_input, outputs=outputs)
    model.compile(optimizer=optimizer,
                  loss='mse')
    
    return model

In [11]:
def generate_model_v4():
    image_input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1))
    
    img = Conv2D(32, (3, 3), padding='same', activation='relu', trainable=False)(image_input)
    img = Conv2D(32, (3, 3), padding='same', activation='relu', trainable=False)(img)
    
    img = MaxPooling2D()(img)
    img = Conv2D(64, (3,3), padding='same', activation='relu', trainable=False)(img)
    
    img = MaxPooling2D()(img)
    img = Conv2D(92, (3,3), padding='same', activation='relu', trainable=False)(img)
    
    img = MaxPooling2D()(img)
    img = Conv2D(128, (3,3), padding='same', activation='relu')(img)
    
    img = Conv2DTranspose(128, (3,3), padding='same', activation='relu')(img)
    img = UpSampling2D()(img)
    
    img = Conv2DTranspose(92, (3,3), padding='same', activation='relu', trainable=False)(img)
    img = UpSampling2D()(img)
    
    img = Conv2DTranspose(64, (3,3), padding='same', activation='relu', trainable=False)(img)
    img = UpSampling2D()(img)
    
    img = Conv2DTranspose(32, (3, 3), padding='same', activation='relu', trainable=False)(img)
    img = Conv2DTranspose(32, (3, 3), padding='same', activation='relu', trainable=False)(img)
    img = Conv2DTranspose(1, (3, 3), padding='same', activation='sigmoid', trainable=False)(img)

    outputs = img

    optimizer = RMSprop()
    model = Model(inputs=image_input, outputs=outputs)
    model.compile(optimizer=optimizer,
                  loss='mse')
    
    return model

In [12]:
def generate_model_v4():
    image_input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1))
    
    img = Conv2D(32, (3, 3), padding='same', activation='relu', trainable=False)(image_input)
    img = Conv2D(32, (3, 3), padding='same', activation='relu', trainable=False)(img)
    
    img = MaxPooling2D()(img)
    img = Conv2D(64, (3,3), padding='same', activation='relu', trainable=False)(img)
    
    img = MaxPooling2D()(img)
    img = Conv2D(92, (3,3), padding='same', activation='relu')(img)
    
    img = MaxPooling2D()(img)
    img = Conv2D(128, (3,3), padding='same', activation='relu', trainable=False)(img)
    
    img = Conv2DTranspose(128, (3,3), padding='same', activation='relu', trainable=False)(img)
    img = UpSampling2D()(img)
    
    img = Conv2DTranspose(92, (3,3), padding='same', activation='relu')(img)
    img = UpSampling2D()(img)
    
    img = Conv2DTranspose(64, (3,3), padding='same', activation='relu', trainable=False)(img)
    img = UpSampling2D()(img)
    
    img = Conv2DTranspose(32, (3, 3), padding='same', activation='relu', trainable=False)(img)
    img = Conv2DTranspose(32, (3, 3), padding='same', activation='relu', trainable=False)(img)
    img = Conv2DTranspose(1, (3, 3), padding='same', activation='sigmoid', trainable=False)(img)

    outputs = img

    optimizer = RMSprop()
    model = Model(inputs=image_input, outputs=outputs)
    model.compile(optimizer=optimizer,
                  loss='mse')
    
    return model

In [ ]:
print(generate_model_v4().summary())

In [ ]:
from keras.callbacks import Callback
from keras import backend as K

class TensorBoard(Callback):

    def __init__(self, log_dir='./logs',
                 write_graph=False,
                 start_steps=0,
                 batch_freq=1):
        super(TensorBoard, self).__init__()
        
        global tf, projector
        import tensorflow as tf
        from tensorflow.contrib.tensorboard.plugins import projector
        
        self.log_dir = log_dir
        self.batch_freq = batch_freq
        self.write_graph = write_graph
        
        self.start_steps = start_steps
        self.steps_counter = 1

    def set_model(self, model):
        self.model = model
        self.sess = K.get_session()
        
        self.merged = tf.summary.merge_all()

        if self.write_graph:
            self.writer = tf.summary.FileWriter(self.log_dir,
                                                self.sess.graph)
        else:
            self.writer = tf.summary.FileWriter(self.log_dir)
            
    def save_scalar(self, logs):
        log = logs or {}
        
        for name, value in logs.items():
            if name in ['batch', 'size']:
                continue
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.writer.add_summary(
                summary,
                self.start_steps + self.steps_counter
            )
        self.writer.flush()
            
    def on_batch_end(self, batch, logs=None):
        if self.steps_counter % self.batch_freq == 0:
            self.save_scalar(logs)
        self.steps_counter += 1

    def on_epoch_end(self, epoch, logs=None):
        self.save_scalar(logs)        

    def on_train_end(self, _):
        self.writer.close()

class ModelCheckpoint(Callback):

    def __init__(self,
                 filepath,
                 start_steps=0,
                 batch_freq=1):
        super(ModelCheckpoint, self).__init__()
        
        self.filepath = filepath
        
        self.steps_counter = 0
        self.start_steps = start_steps
        self.batch_freq = batch_freq
        
        self.steps_counter
        
    def save_model(self):
        self.model.save_weights(self.filepath, overwrite=True)
        
    def on_batch_end(self, batch, logs=None):
        if self.steps_counter % self.batch_freq == 0:
            self.save_model()
        self.steps_counter += 1
        
    def on_epoch_end(self, epoch, logs=None):
        self.save_model()

Model Migration

In [ ]:
# model_v1 = generate_model_v1()
# model_v1.load_weights('./model/model-ae-weight-1.hdf5')
# model_v1_weights = model_v1.get_weights()

# model_v2 = generate_model_v2()

# for i in range(1,3):
#     model_v2.layers[i].set_weights(
#         model_v1.layers[i].get_weights()
#     )
    
# for i in range(-1,-3,-1):
#     model_v2.layers[i].set_weights(
#         model_v1.layers[i].get_weights()
#     )

# del model_v1
# model = model_v2

In [ ]:
# model_v2 = generate_model_v2()
# model_v2.load_weights('./model/model-ae-weight-2.hdf5')
# model_v2_weights = model_v2.get_weights()

# model_v3 = generate_model_v3()

# for i in range(1,5):
#     print(model_v3.layers[i].name)
#     model_v3.layers[i].set_weights(
#         model_v2.layers[i].get_weights()
#     )
    
# for i in range(-1,-5,-1):
#     print(model_v3.layers[i].name)
#     model_v3.layers[i].set_weights(
#         model_v2.layers[i].get_weights()
#     )

# del model_v2
# model = model_v3

In [ ]:
model_v3 = generate_model_v3()
model_v3.load_weights('./model/model-ae-weight-3.hdf5')
model_v3_weights = model_v3.get_weights()

model_v4 = generate_model_v4()

for i in range(1,7):
    print(model_v3.layers[i].name)
    model_v4.layers[i].set_weights(
        model_v3.layers[i].get_weights()
    )
print(" ")
    
for i in range(-6,0):
    print(model_v3.layers[i].name)
    model_v4.layers[i].set_weights(
        model_v3.layers[i].get_weights()
    )

del model_v3
model = model_v4

In [ ]:
model_v4 = generate_model_v4_2()
model_v4.load_weights('./model/model-ae-weight-4.hdf5')
model = model_v4

Train Model

In [ ]:
model.fit_generator(
    input_generator(x_train_idx),
    steps_per_epoch=steps_per_epoch,
    validation_data=input_generator(x_test_idx),
    validation_steps=validation_steps,
    max_queue_size=3,
    epochs=1,
    use_multiprocessing=True,
    callbacks=[
        TensorBoard(
            log_dir="./model/logs/",
            batch_freq=5
        ),
        ModelCheckpoint(
            filepath="./model/model-ae-weight-5.hdf5",
            batch_freq=60
        )
    ]
)

In [ ]:
print("!")

In [13]:
model_v4 = generate_model_v4()
model_v4.load_weights('./model/model-ae-weight-4.hdf5')
model = model_v4

In [ ]:
for x,y in input_generator(x_test_idx):
    print(y[0].reshape(IMAGE_SIZE * IMAGE_SIZE)[0:50])
    for i in range(x.shape[0]):
        xs = x[i].reshape((IMAGE_SIZE, IMAGE_SIZE))
        ys = model.predict(xs.reshape((1, IMAGE_SIZE, IMAGE_SIZE, 1)))
        ys = ys[0].reshape((IMAGE_SIZE, IMAGE_SIZE))
        
        yss = y[i].reshape((IMAGE_SIZE, IMAGE_SIZE))

#         plt.imshow(xs)
#         plt.show()
#         plt.imshow(ys)
#         plt.show()
        
        f, axarr = plt.subplots(1,3,figsize=(15,85))
        axarr[0].imshow(xs, cmap='Greys')
        axarr[1].imshow(yss, cmap='Greys')
        axarr[2].imshow(ys, cmap='Greys')
    break

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
